In [1]:
import re
import pandas as pd
import numpy as np
import simplemma
from stop_words import get_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
stop_words = get_stop_words('uk')

In [28]:
#попередня обробка тексту
def ClearText(text):
    #переведення до нижнього регістру всіх слів
    cleartext = text.lower()
    #print(cleartext)
    #прибирання пустих рядків та розрив рядків
    cleartext = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', cleartext) 
    #залишаємо лише слова, прибираємо пунктуацію та числа
    cleartext = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', cleartext) #deleting symbols  
    #cleartext = cleartext.translate(remove_digits)
    cleartext = cleartext.replace("\\", "")
    cleartext = cleartext.rstrip()
    #прибираємо зайві пробіи
    cleartext = re.sub(" +", " ", cleartext)
    #ділимо речення на список слів, розбиваємо по пробілам
    cleartext = re.split(" ", cleartext)
    #прибираємо стопслова
    cleartext = [word for word in cleartext if word not in stop_words]
    #прибираємо слова, довжина який менше 3 букв
    cleartext = [word for word in cleartext if len(word) > 3]
    #лематизація слів
    cleartext = [simplemma.lemmatize(word, lang='uk') for word in cleartext]
    return ' '.join(cleartext)

In [97]:
print(stop_words)

['a', 'б', 'в', 'г', 'е', 'ж', 'з', 'м', 'т', 'у', 'я', 'є', 'і', 'аж', 'ви', 'де', 'до', 'за', 'зі', 'ми', 'на', 'не', 'ну', 'нх', 'ні', 'по', 'та', 'ти', 'то', 'ту', 'ті', 'це', 'цю', 'ця', 'ці', 'чи', 'ще', 'що', 'як', 'їй', 'їм', 'їх', 'її', 'або', 'але', 'ало', 'без', 'був', 'вам', 'вас', 'ваш', 'вже', 'все', 'всю', 'вся', 'від', 'він', 'два', 'дві', 'для', 'ким', 'мож', 'моя', 'моє', 'мої', 'міг', 'між', 'мій', 'над', 'нам', 'нас', 'наш', 'нею', 'неї', 'них', 'ніж', 'ній', 'ось', 'при', 'про', 'під', 'пір', 'раз', 'рік', 'сам', 'сих', 'сім', 'так', 'там', 'теж', 'тим', 'тих', 'той', 'тою', 'три', 'тут', 'хоч', 'хто', 'цей', 'цим', 'цих', 'час', 'щоб', 'яка', 'які', 'адже', 'буде', 'буду', 'будь', 'була', 'були', 'було', 'бути', 'вами', 'ваша', 'ваше', 'ваші', 'весь', 'вниз', 'вона', 'вони', 'воно', 'всею', 'всім', 'всіх', 'втім', 'геть', 'далі', 'двох', 'день', 'дуже', 'зате', 'його', 'йому', 'каже', 'кого', 'коли', 'кому', 'крім', 'куди', 'лише', 'люди', 'мало', 'мати', 'мене', 

In [74]:
df_train = pd.read_excel('C:/Users/helen/OneDrive/Рабочий стол/пары/NLP_train.xlsx')
df_test = pd.read_excel('C:/Users/helen/OneDrive/Рабочий стол/пары/NLP_test.xlsx')
df_train.head()

,Comment,Category
0,Що вам найбільше сподобалося в перевтіленні Лю...,Not_Toxic
1,"яка гарна родина, щастя вам))",Not_Toxic
2,Дякую команді за такий чудовий проєкт. На жаль...,Not_Toxic
3,"Щодо танго не згодна. Виглядає так, наче Люда ...",Not_Toxic
4,"Людмила дуже гарна жінка, їй дуже пасує жіночн...",Not_Toxic


In [59]:
df = pd.read_excel('C:/Users/helen/OneDrive/Рабочий стол/пары/NLP_all.xlsx')
df = df.sample(frac=1) 
df

,Comment,Category
553,Шоу для недалеких. Нас у школі вчили зовсім ін...,Toxic
1001,"Добрий вечір.Ірпінь, будь ласка, гляньте.",Not_Toxic
598,"Яка ж крінжуха, шоу відстій",Toxic
1108,"Потрібно ""Коли ми вдома"" перекладати та продав...",Not_Toxic
378,"привіт, новий підписчи, виглядаєш дуже жіночо",Not_Toxic
...,...,...
899,Super. Дуже чудовий дует.\nДякую за гарний нас...,Not_Toxic
890,"Бережи Боже монтажера, це просто вогонь, обожн...",Not_Toxic
836,Дякую за це відео. Воно надихає на дії,Not_Toxic
795,Добрий день! Чи немає у Вас PayPal аккаунта? Б...,Not_Toxic


In [60]:
df_train = df.iloc [:round(df.shape[0]*0.75)]
df_test = df.iloc [round(df.shape[0]*0.75):]

In [61]:
df_train['ClearText'] = df_train.apply(lambda x: ClearText(x['Comment']), axis=1)
df_test['ClearText'] = df_test.apply(lambda x: ClearText(x['Comment']), axis=1)

C:\Users\helen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\helen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
df_test

,Comment,Category,ClearText
869,Світлана дуже позитивна і щаслива. Мрії збуваю...,Not_Toxic,Світлана позитивний щасливий мрія збуваються о...
168,кузьма разом з притулою точно б зібрали на яде...,Not_Toxic,Кузьма раз притулою точно зібрати ядерний бомба
160,Світла память.,Not_Toxic,світло память
1088,Ви несповна розуму....,Toxic,несповна розум
14,Будь-який коментар :),Not_Toxic,коментар
...,...,...,...
499,"Все, що гарне у Львові, збудували поляки!",Toxic,гарний Львів збудувати поляк
976,Дуже гарне відео ️,Not_Toxic,гарний відео
297,Супер супер супер класс,Not_Toxic,супер супер супер класс
379,"Туреччина така ж, як і Росія, навіть гірша, ко...",Toxic,Туреччина росія гірший йтися злочин людяності ...


In [62]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df_train['ClearText'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# MultinomialNB

In [63]:
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])
text_clf = text_clf.fit(df_train['ClearText'], df_train['Category'])

In [64]:
predicted = text_clf.predict(df_test['ClearText'])
np.mean(predicted == df_test['Category'])

0.6804123711340206

# SVM

In [65]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3,  random_state=42)),
])
train_svm = text_clf_svm.fit(df_train['ClearText'], df_train['Category'])

In [66]:
predicted_svm = text_clf_svm.predict(df_test['ClearText'])
np.mean(predicted_svm == df_test['Category'])

0.718213058419244

In [67]:
predicted_svm

array(['Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Toxic', 'Not_Toxic', 'Toxic', 'Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Toxic', 'Not_Toxic', 'Not_Toxic', '

# LogisticRegression

In [68]:
text_clf_log_reg = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)),
])
train_log_reg = text_clf_log_reg.fit(df_train['ClearText'], df_train['Category'])

[LibLinear]

In [72]:
predicted_log_reg = text_clf_log_reg.predict(df_test['ClearText'])
np.mean(predicted_log_reg == df_test['Category'])

0.7250859106529209

In [70]:
predicted_log_reg

array(['Not_Toxic', 'Not_Toxic', 'Toxic', 'Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Toxic', 'Not_Toxic', 'Toxic', 'Toxic', 'Not_Toxic', 'Toxic',
       'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic', 'Not_Toxic',
       'Toxic', 'Not_Toxic', 'Not_Toxic', '

In [88]:
predicted_log_reg_df = pd.DataFrame(predicted_log_reg, columns=['predictions'])
df_test.index = range(0, df_test.shape[0])
result = pd.concat([predicted_log_reg_df, df_test], axis=1)
result.to_csv('C:/Users/helen/OneDrive/Рабочий стол/пары/prediction.csv')
result

,predictions,Comment,Category,ClearText
0,Not_Toxic,ПАРАША ЗІ СМАКОМ СОЛЯРКИ ..І ГУМОР НИЖЧЕ ПЛІНТ...,Toxic,параша смак солярки гумор плінтуса
1,Not_Toxic,Світла пам'ять Руслані Писанки,Not_Toxic,світло пам'ять Руслана писанка
2,Toxic,"Складно мені повірити, що щось зміниться",Toxic,складно повірити щось змінитися
3,Toxic,"Серёжа,розкажи як ты Людину ножом зарізав?",Toxic,серёжа розказати людина ножом зарізати
4,Not_Toxic,Коли Садовий фасади поремонтує тих обдертих бу...,Toxic,садовий фасад поремонтує обдертих будинкiв
...,...,...,...,...
286,Not_Toxic,Super. Дуже чудовий дует.\nДякую за гарний нас...,Not_Toxic,super чудовий дует \nдякую гарний настрій \nслава
287,Not_Toxic,"Бережи Боже монтажера, це просто вогонь, обожн...",Not_Toxic,берегти бог монтажера вогонь обожнюю
288,Not_Toxic,Дякую за це відео. Воно надихає на дії,Not_Toxic,відео надихає
289,Not_Toxic,Добрий день! Чи немає у Вас PayPal аккаунта? Б...,Not_Toxic,добрити paypal аккаунта зручно скинути гроші
